In [23]:
#Imports
from scripts import bookdatafunctions as bdf
from scripts import corpusMLfunctions as cmf
import json

In [24]:
#Constants
AGE_SHEET = "ISBN_MAPS/ISBN2AGE.xlsx"
AUTH_SHEET = "ISBN_MAPS/ISBN2AUTH.xlsx"
CONLLUS_FOLDER = "Conllus"
SNIPPET_LENS = [5,10,25,50,75,100]

In [25]:
#Load corpus
corpus = bdf.mapGroup2Age(bdf.maskPropn(bdf.initBooksFromConllus(CONLLUS_FOLDER)), AGE_SHEET)

In [26]:
#Generate train-test-eval split for keys
author_level_split = cmf.splitOnAuthorLevel(list(corpus.keys()), AUTH_SHEET)
train_target, test_target, eval_target = cmf.generateAgeStratificationAmounts(corpus, 0.7)
print(train_target)
train_keys, test_keys, eval_keys = cmf.doTrainTestEvalSplitSeriesLevel(author_level_split, train_target, test_target, eval_target)

{6: 11, 5: 10, 11: 6, 10: 9, 16: 2, 9: 35, 7: 25, 14: 32, 18: 0, 15: 19, 8: 21, 12: 18, 13: 15}


In [27]:
def checkDuplicates(trainkeys: list, testkeys:list):
    for key in trainkeys:
        if key in testkeys:
            return False
    return True

def checkGenres(trainkeys: list, correct_amounts: dict):
    amounts = {1:0, 2:0, 3:0}
    for key in trainkeys:
        genre = int(key[-1])
        amounts[genre] = amounts[genre] + 1
    for key in correct_amounts:
        if abs(correct_amounts[key]-amounts[key]) > 1:
            return False
    return True

In [ ]:
trainkeys_straps = []
testkeys_straps = []
evalkeys_straps = []

while len(trainkeys_straps) != 100:
    train_keys, test_keys, eval_keys = cmf.doTrainTestEvalSplitSeriesLevel(author_level_split, train_target, test_target, eval_target)
    if len(train_keys) == 210 and checkDuplicates(train_keys, test_keys+eval_keys) and checkGenres(train_keys, {1:158, 2:32, 3:20}):
        print(len(trainkeys_straps))
        trainkeys_straps.append(train_keys)
        testkeys_straps.append(test_keys)
        evalkeys_straps.append(eval_keys)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [28]:
counter = 0
for x in trainkeys_straps:
    for y in trainkeys_straps:
        if x==y:
            counter += 1
print(counter)

100


In [29]:
temp_dict = []
for i in range(len(trainkeys_straps)):
    temp_dict.append({'id':i, 'train_keys':trainkeys_straps[i], 'eval_keys':evalkeys_straps[i], 'test_keys':testkeys_straps[i]})
with open("Keylists.jsonl", 'w') as f:
    f.write('\n'.join(map(json.dumps, temp_dict)))